In [15]:
import os

os.environ['GRB_LICENSE_FILE'] = '/Users/charlottelee/gurobi.lic'

import gurobipy as gr

print(gr.gurobi.version())

license_path = os.getenv('GRB_LICENSE_FILE')
print("License file path:", license_path)


(12, 0, 2)
License file path: /Users/charlottelee/gurobi.lic


# Dataset Analysis

In [16]:
import pickle
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from collections import Counter, defaultdict

def dataset_analysis(cap_file='cap.pck', pref_file='pref.pck', verbose=True):
    # load data
    with open(cap_file, 'rb') as f:
        capacities = pickle.load(f)
    with open(pref_file, 'rb') as f:
        preferences = pickle.load(f)

    if verbose:
        print("=== BASIC DATASET STATISTICS ===")
    school_ids = list(capacities.keys())
    all_ids = set(preferences.keys())
    student_ids = [id for id in all_ids if id not in capacities]
    
    if verbose:
        print(f"School tracks: {len(school_ids)}")
        print(f"Students: {len(student_ids)}")
        print(f"Total capacity: {sum(capacities.values())}")
        print(f"Capacity/student ratio: {sum(capacities.values())/len(student_ids):.2f}")
    
    if verbose:
        print("\n=== SCHOOL STRUCTURE ANALYSIS ===")
    # school suffixes (PRI, REG, PIE)
    suffixes = [s.split('_')[-1] if '_' in s else "UNKNOWN" for s in school_ids]
    suffix_counter = Counter(suffixes)
    if verbose:
        print(f"Track types: {dict(suffix_counter)}")
    
    # organize schools by base name (removing suffix)
    base_schools = defaultdict(list)
    for school_id in school_ids:
        if '_' in school_id:
            parts = school_id.split('_')
            suffix = parts[-1]
            base_name = '_'.join(parts[:-1])
            base_schools[base_name].append((school_id, suffix))
    
    if verbose:
        print(f"Unique base schools: {len(base_schools)}")
    
    # count tracks per base school
    track_counts = Counter([len(tracks) for _, tracks in base_schools.items()])
    if verbose:
        print(f"Tracks per base school: {dict(track_counts)}")
    
    # schools with PIE tracks
    pie_schools = [base for base, tracks in base_schools.items() 
                  if any(suffix == 'PIE' for _, suffix in tracks)]
    if verbose:
        print(f"\nSchools with PIE tracks ({len(pie_schools)}):")
    for base in pie_schools:
        tracks = base_schools[base]
        if verbose:
            print(f"  {base}: {[suffix for _, suffix in tracks]}")
        for school_id, suffix in tracks:
            if verbose:
                print(f"    {suffix}: capacity {capacities[school_id]}")

    if verbose:
        print("\n=== SCHOOL CAPACITY DISTRIBUTION ===")
    pri_capacities = [cap for s, cap in capacities.items() if s.endswith('_PRI')]
    reg_capacities = [cap for s, cap in capacities.items() if s.endswith('_REG')]
    pie_capacities = [cap for s, cap in capacities.items() if s.endswith('_PIE')]

    if verbose:
        print(f"PRI tracks: {len(pri_capacities)}, total capacity: {sum(pri_capacities)}")
        print(f"REG tracks: {len(reg_capacities)}, total capacity: {sum(reg_capacities)}")
        print(f"PIE tracks: {len(pie_capacities)}, total capacity: {sum(pie_capacities)}")
    
    if verbose:
        print("\n=== STUDENT PREFERENCE ANALYSIS ===")
    student_prefs = {s: preferences[s] for s in student_ids}
    
    # count preferences per student
    pref_counts = [len(prefs) for s, prefs in student_prefs.items()]
    if verbose:
        print(f"Avg preferences per student: {np.mean(pref_counts):.1f}")
        print(f"Min: {min(pref_counts)}, Max: {max(pref_counts)}")
    
    # students' preference distribution across the schools
    all_preferences = []
    for s, prefs in student_prefs.items():
        for _, school in prefs.items():
            all_preferences.append(school)
    
    school_popularity = Counter(all_preferences)
    
    # most popular schools
    if verbose:
        print("\nTop 20 most popular schools (across all preference ranks):")
    for school, count in school_popularity.most_common(20):
        capacity = capacities.get(school, 0)
        if verbose:
            print(f"{school}: {count} students ranked it (capacity: {capacity}, ratio: {count/capacity:.1f}x)")
    
    # preference distribution by rank
    preferences_by_rank = defaultdict(Counter)
    for s, prefs in student_prefs.items():
        for rank, school in prefs.items():
            preferences_by_rank[int(rank)][school] += 1
    
    # most first-choice schools, count preference/capcity ratio
    if verbose:
        print("\nTop 10 first-choice schools:")
    for school, count in preferences_by_rank[1].most_common(10):
        capacity = capacities.get(school, 0)
        if verbose:
            print(f"{school}: {count} first choices (capacity: {capacity}, ratio: {count/capacity:.1f}x)")
    
    if verbose:
        print("\n=== SCHOOL PRIORITY ANALYSIS ===")
    school_priorities = {s: preferences[s] for s in school_ids if s in preferences}
    
    # school's priority list length
    priority_counts = [len(priorities) for s, priorities in school_priorities.items()]
    if verbose:
        print(f"Avg students in priority list: {np.mean(priority_counts):.1f}")
        print(f"Min: {min(priority_counts)}, Max: {max(priority_counts)}")
    
    # check for empty priority lists
    empty_priority = [s for s, priorities in school_priorities.items() if len(priorities) == 0]
    if verbose:
        print(f"Schools with empty priority lists: {len(empty_priority)}")
    if empty_priority:
        if verbose:
            print(f"Examples: {empty_priority[:5]}")
    
    # compare same school (PRI vs REG) priority lists
    same_priorities = 0
    different_priorities = 0
    
    for base, tracks in base_schools.items():
        track_ids = [id for id, _ in tracks]
        priority_lists = {id: school_priorities.get(id, {}) for id in track_ids if id in school_priorities}
        
        if len(priority_lists) < 2:
            continue
        
        # compare all pairs of priority lists
        track_ids = list(priority_lists.keys())
        all_same = True
        
        for i in range(len(track_ids)):
            for j in range(i+1, len(track_ids)):
                list1 = list(priority_lists[track_ids[i]].values())
                list2 = list(priority_lists[track_ids[j]].values())
                
                if list1 != list2:
                    all_same = False
                    break
        
        if all_same:
            same_priorities += 1
        else:
            different_priorities += 1
    
    if verbose:
        print(f"Schools with identical priority lists across tracks: {same_priorities}")
        print(f"Schools with different priority lists across tracks: {different_priorities}")
    
    if verbose:
        print("\n=== POTENTIAL STABILITY ISSUES ===")
    
    # demand-supply ratio
    demand_supply = {}
    for school in school_ids:
        demand = school_popularity.get(school, 0)
        supply = capacities.get(school, 0)
        if supply > 0:
            ratio = demand / supply
            demand_supply[school] = ratio
    
    extreme_schools = [(s, ratio) for s, ratio in demand_supply.items() if ratio > 10]
    extreme_schools.sort(key=lambda x: x[1], reverse=True)
    
    if verbose:
        print(f"Schools with extreme demand-supply ratio (>10x):")
    for school, ratio in extreme_schools[:10]:
        demand = school_popularity.get(school, 0)
        supply = capacities.get(school, 0)
        if verbose:
            print(f"{school}: {demand} students prefer, capacity {supply}, ratio {ratio:.1f}x")
    

    # how many students have unique first choices
    first_choices = [next(iter(prefs.values())) for s, prefs in student_prefs.items() if prefs]
    unique_first = len(set(first_choices))
    total_first = len(first_choices)
    if verbose:
        print(f"\nStudents with unique first choices: {unique_first}/{total_first} ({unique_first/total_first*100:.1f}%)")


    # calculate preference correlation 
    
    
    # find valid student-school pairs
    valid_pairs = []
    for student_id, prefs in student_prefs.items():
        if isinstance(prefs, dict):
            for rank, school_id in prefs.items():
                if school_id in school_priorities:
                    if student_id in school_priorities[school_id].values():
                        valid_pairs.append((student_id, school_id))
    if verbose:
        print(f"\nNumber of valid (student, school) pairs: {len(valid_pairs)}")


    # for further use
    return {
        'student_ids': student_ids,
        'school_ids': school_ids,
        'school_capacities': capacities,
        'preferences': preferences,
        'student_preferences': student_prefs,
        'school_priorities': school_priorities,
        'base_schools': base_schools,
        'demand_supply': demand_supply,
        'preferences_by_rank': preferences_by_rank,
        'extreme_schools': extreme_schools,
        'valid_pairs': valid_pairs
    }


In [17]:
dataset_analysis();

=== BASIC DATASET STATISTICS ===
School tracks: 100
Students: 1395
Total capacity: 1624
Capacity/student ratio: 1.16

=== SCHOOL STRUCTURE ANALYSIS ===
Track types: {'REG': 49, 'PRI': 49, 'PIE': 2}
Unique base schools: 49
Tracks per base school: {2: 47, 3: 2}

Schools with PIE tracks (2):
  8417_401000000133: ['REG', 'PIE', 'PRI']
    REG: capacity 10
    PIE: capacity 2
    PRI: capacity 3
  8437_401000000133: ['REG', 'PIE', 'PRI']
    REG: capacity 23
    PIE: capacity 2
    PRI: capacity 5

=== SCHOOL CAPACITY DISTRIBUTION ===
PRI tracks: 49, total capacity: 268
REG tracks: 49, total capacity: 1352
PIE tracks: 2, total capacity: 4

=== STUDENT PREFERENCE ANALYSIS ===
Avg preferences per student: 6.5
Min: 2, Max: 32

Top 20 most popular schools (across all preference ranks):
24307_401000000232_PRI: 515 students ranked it (capacity: 10, ratio: 51.5x)
24307_401000000232_REG: 515 students ranked it (capacity: 54, ratio: 9.5x)
11680_401000000133_PRI: 368 students ranked it (capacity: 4, 

# Baseline Model: Quadratic-Constrained

In [18]:
import gurobipy as gp
from gurobipy import GRB
import time


def create_quadratic_model(data, B, valid_pairs):
    """
    Creates the baseline quadratic-constrained model.
    
    Parameters:
    - data: Dictionary containing dataset information
    - B: Budget for capacity expansion
    - valid_pairs: List of valid (student, school) pairs

    """
    students = data['student_ids']
    schools = data['school_ids']
    capacities = data['school_capacities']
    student_preferences = data['student_preferences']
    school_priorities = data['school_priorities']
    
    # calculate preference weights
    w = {}
    for s in students:
        prefs = student_preferences.get(s, {})
        n = len(prefs)  
        for c in schools:
            if c in prefs.values():
                # find which rank this school has for this student
                for rank, school in prefs.items():
                    if school == c:
                        w[s, c] = n - int(rank) + 1
                        break
            else:
                w[s, c] = 0
    

    model = gp.Model("QuadraticStableMatching")
    model.Params.OutputFlag = 0  # suppress solver output
    
    # decision variables
    x = model.addVars(valid_pairs, vtype=GRB.BINARY, name="x")
    t = model.addVars(schools, vtype=GRB.INTEGER, lb=0, name="t")
    
    # store variables
    model._x = x
    model._t = t
    
    # Objective function: maximize preference satisfaction
    model.setObjective(
        gp.quicksum(w[s, c] * x[s, c] for (s, c) in valid_pairs),
        GRB.MAXIMIZE  
    )

    # Constraint 1: Each student assigned to at most one school
    for s in students:
        model.addConstr(
            gp.quicksum(x[s, c] for (s2, c) in valid_pairs if s2 == s) <= 1,
            name=f"StudentLimit_{s}"
        )

    # Constraint 2: Each school's capacity
    for c in schools:
        model.addConstr(
            gp.quicksum(x[s, c2] for (s, c2) in valid_pairs if c2 == c) <= capacities[c] + t[c],
            name=f"Capacity_{c}"
        )

    # Constraint 3: Total expansion limited by budget
    model.addConstr(
        gp.quicksum(t[c] for c in schools) <= B,
        name="TotalExpansion"
    )
    
    # Constraint 4: Stability constraints (quadratic form)
    for s in students:
        s_prefs = student_preferences.get(s, {})
        
        # for each school in this student's preferences
        for s_rank, c in s_prefs.items():
            if (s, c) not in valid_pairs:
                continue
            
            # find schools that student s prefers more than c
            better_schools_for_s = [
                sch for r, sch in s_prefs.items() 
                if int(r) < int(s_rank) and (s, sch) in valid_pairs
            ]
            
            # find students that school c prioritizes over student s
            if c in school_priorities and s in school_priorities[c]:
                c_priorities = school_priorities[c]
                s_pos_in_c = c_priorities.index(s)
                better_students_for_c = [
                    sp for sp in c_priorities[:s_pos_in_c] 
                    if (sp, c) in valid_pairs
                ]
                
                # var for "school c filled with higher priority students"
                is_filled_var = model.addVar(vtype=GRB.BINARY, name=f"filled_{s}_{c}")
                
                if better_students_for_c:
                    # higher priority students assigned to c
                    higher_priority_assigned = gp.quicksum(
                        x[sp, c] for sp in better_students_for_c
                    )
                    
                    # set is_filled_var = 1 if higher_priority_assigned ≥ capacity
                    big_M = len(better_students_for_c)
                    
                    # if is_filled_var = 1, then school is filled with higher priority students
                    model.addConstr(
                        higher_priority_assigned >= (capacities[c] + t[c]) - big_M * (1 - is_filled_var),
                        name=f"FilledIf_{s}_{c}"
                    )
                    
                    # if school is not filled with higher priority students, then is_filled_var = 0
                    model.addConstr(
                        higher_priority_assigned <= (capacities[c] + t[c] - 1) + big_M * is_filled_var,
                        name=f"NotFilledIf_{s}_{c}"
                    )
                else:
                    # no higher priority students, so this condition can't be satisfied
                    model.addConstr(is_filled_var == 0, name=f"NoHigherPriority_{s}_{c}")
                
                # Main stability constraint: Either:
                # 1. Student s is assigned to school c, OR
                # 2. Student s is assigned to a school they prefer more than c, OR
                # 3. School c is filled with students it prefers more than s
                model.addConstr(
                    x[s, c] + 
                    gp.quicksum(x[s, better] for better in better_schools_for_s) + 
                    is_filled_var >= 1,
                    name=f"Stability_{s}_{c}"
                )
    
    return model

def solve_quadratic_model(data, B, valid_pairs):
    """
    Solves the quadratic model and reports results.
    """
    start_time = time.time()
    model = create_quadratic_model(data, B, valid_pairs)
    model.optimize()

    status = model.Status
    if status == GRB.OPTIMAL:
        print("Optimal solution found.")
    elif status == GRB.SUBOPTIMAL:
        print("Suboptimal solution found.")
    elif status == GRB.INFEASIBLE:
        print("Model is infeasible.")
        return time.time() - start_time, float('inf')
    elif status == GRB.UNBOUNDED:
        print("Model is unbounded.")
        return time.time() - start_time, float('inf')
    else:
        print(f"Solver ended with status code: {status}")
        return time.time() - start_time, float('inf')

    # extract results
    x = model._x
    t = model._t
    
    success_pairs = sum(1 for (_, _), var in x.items() if var.X > 0.5)
    
    print("\n=== Result ===")
    print(f"Success pairs: {success_pairs}")
    
    # show 5 example assignments
    displayed = 0
    for (s, c), var in x.items():
        if var.X > 0.5 and displayed < 5:
            print(f"Student {s} -> School {c}")
            displayed += 1
    
    # count expansions
    total_expansion = sum(var.X for var in t.values())
    expanded_schools = sum(1 for var in t.values() if var.X > 0.5)
    
    print(f"\nExpansion used: {int(total_expansion)}/{B}")
    print(f"Schools expanded: {expanded_schools}")
    
    if expanded_schools > 0:
        print("\nTop 5 expansions:")
        expansions = [(c, var.X) for c, var in t.items() if var.X > 0.5]
        expansions.sort(key=lambda x: x[1], reverse=True)
        for c, exp in expansions[:5]:
            print(f"School {c}: +{int(exp)} seats")
    
    # objective value
    objective_value = model.ObjVal
    print(f"\nObjective Value: {objective_value:.4f}")

    # metrics
    solver_time = time.time() - start_time
    mip_gap = model.MIPGap if model.SolCount > 0 else float('inf')
    
    return solver_time, mip_gap

# if __name__ == "__main__":
#     data = dataset_analysis(verbose=False)
#     valid_pairs = data['valid_pairs']
    
#     budget_values = [0, 25, 50, 75, 100, 125, 150, 175, 200]
    
#     for B in budget_values:
#         print(f"\n==== Testing with Budget B = {B} ====")
#         try:
#             time_taken, gap = solve_quadratic_model(data, B, valid_pairs)
#             print(f"Solver time: {time_taken:.2f} s")
#             print(f"MIP gap: {gap:.4f}")
#         except Exception as e:
#             print(f"Error: {e}")

# Linearization Model 1: Individualized Linearization

In [19]:
def create_linear_model_1(data, B, valid_pairs):
    """Create the linearized model using McCormick envelopes"""
    model = gp.Model("SchoolMatching_Linearized")
    model.Params.OutputFlag = 0  # disable solver output

    students = data['student_ids']
    schools = data['school_ids']
    capacities = data['school_capacities']
    student_prefs = data['student_preferences']
    school_priorities = data['school_priorities']

    # create main decision variables
    x = model.addVars(valid_pairs, vtype=GRB.BINARY, name="assign")  # x[s,c] = 1 if student s assigned to school c
    t = model.addVars(schools, vtype=GRB.INTEGER, lb=0, name="expand")  # t[c] = seats added to school c

    # create preference weights (higher weight for more preferred schools)
    w = {}
    for s in students:
        prefs = student_prefs[s]
        for rank, c in prefs.items():
            if (s, c) in valid_pairs:
                w[s, c] = len(prefs) - int(rank) + 1  # weight by preference rank

    # objective: Maximize total preference weight
    model.setObjective(
        gp.quicksum(w[s, c] * x[s, c] for (s, c) in valid_pairs),
        GRB.MAXIMIZE
    )

    # constraint: Each student assigned to at most one school
    for s in students:
        model.addConstr(
            gp.quicksum(x[s, c] for c in schools if (s, c) in valid_pairs) <= 1,
            name=f"one_assignment_{s}"
        )

    # constraint: School capacity limits
    for c in schools:
        model.addConstr(
            gp.quicksum(x[s, c] for s in students if (s, c) in valid_pairs) <= capacities[c] + t[c],
            name=f"capacity_{c}"
        )

    # constraint: Total expansion budget
    model.addConstr(
        gp.quicksum(t[c] for c in schools) <= B,
        name="total_expansion"
    )

    # auxiliary variables for linearization
    M = B  # upper bound for t[c]
    alpha = model.addVars(valid_pairs, vtype=GRB.INTEGER, lb=0, name="alpha")  # α[s,c] = x[s,c] * t[c]
    
    # McCormick envelopes for α[s,c] = x[s,c] * t[c]
    for (s, c) in valid_pairs:
        model.addConstr(alpha[s, c] <= M * x[s, c], name=f"mccormick1_{s}_{c}")
        model.addConstr(alpha[s, c] <= t[c], name=f"mccormick2_{s}_{c}")
        model.addConstr(alpha[s, c] >= t[c] - M * (1 - x[s, c]), name=f"mccormick3_{s}_{c}")
        model.addConstr(alpha[s, c] >= 0, name=f"mccormick4_{s}_{c}")

    # stability constraints
    for s in students:
        prefs = student_prefs[s]
        for rank, c in prefs.items():
            if (s, c) not in valid_pairs:
                continue
            
            # schools preferred over c by student s
            better_schools = [sch for r, sch in prefs.items() 
                            if int(r) < int(rank) and (s, sch) in valid_pairs]
            
            # students prioritized over s by school c
            if c in school_priorities and s in school_priorities[c]:
                priority_list = school_priorities[c]
                s_pos = priority_list.index(s)
                higher_priority = priority_list[:s_pos]
                
                # var for "school filled with higher priority students"
                filled = model.addVar(vtype=GRB.BINARY, name=f"filled_{s}_{c}")
                
                # linearize (capacity + t) * filled
                gamma = model.addVar(vtype=GRB.INTEGER, lb=0, name=f"gamma_{s}_{c}")
                model.addConstr(gamma <= M * filled, name=f"gamma1_{s}_{c}")
                model.addConstr(gamma <= t[c], name=f"gamma2_{s}_{c}")
                model.addConstr(gamma >= t[c] - M * (1 - filled), name=f"gamma3_{s}_{c}")
                
                # constraint: filled = 1 if school filled with higher priority students
                model.addConstr(
                    gp.quicksum(x[sp, c] for sp in higher_priority if (sp, c) in valid_pairs) + 
                    capacities[c] * filled + gamma >= capacities[c] + t[c],
                    name=f"fill_condition_{s}_{c}"
                )

                model.addConstr(
                    x[s, c] + gp.quicksum(x[s, sch] for sch in better_schools) + filled >= 1,
                    name=f"stability_{s}_{c}"
                )

    return model

def solve_linear_model_1(data, B, valid_pairs):
    """Solve the model and return results"""
    start_time = time.time()
    model = create_linear_model_1(data, B, valid_pairs)
    model.optimize()
    
    result = {
        'status': model.Status,
        'time': time.time() - start_time,
        'gap': model.MIPGap if model.SolCount > 0 else float('inf'),
        'objective': model.ObjVal if model.SolCount > 0 else None,
        'assignments': [],
        'expansions': {}
    }
    
    if model.SolCount > 0:
        x = model.getAttr('X', model.getVars())
        for i, var in enumerate(model.getVars()):
            if 'assign[' in var.VarName and x[i] > 0.5:
                s, c = var.VarName.split('[')[1].split(']')[0].split(',')
                result['assignments'].append((s.strip("' "), c.strip("' ")))
        
        for c in data['school_ids']:
            t_var = model.getVarByName(f"expand[{c}]")
            if t_var and t_var.X > 0.5:
                result['expansions'][c] = t_var.X


    solver_time = time.time() - start_time
    mip_gap = model.MIPGap if model.SolCount > 0 else float('inf')
    
    return solver_time, mip_gap


# Linearization Model 2: Aggregated

In [20]:
def create_linear_model_2(data, B, valid_pairs):
    """
    Creates linearization model 2 with aggregated linearization.
    """
    students = data['student_ids']
    schools = data['school_ids']
    capacities = data['school_capacities']
    student_preferences = data['student_preferences']
    school_priorities = data['school_priorities']
    
    # calculate preference weights
    w = {}
    for s in students:
        prefs = student_preferences.get(s, {})
        n = len(prefs)  
        for c in schools:
            if c in prefs.values():
                # find which rank this school has for this student
                for rank, school in prefs.items():
                    if school == c:
                        w[s, c] = n - int(rank) + 1
                        break
            else:
                w[s, c] = 0
    
    model = gp.Model("LinearizedModel2")
    model.Params.OutputFlag = 0  # suppress solver output
    
    # decision variables
    x = model.addVars(valid_pairs, vtype=GRB.BINARY, name="x")
    t = model.addVars(schools, vtype=GRB.INTEGER, lb=0, name="t")
    
    # store variables
    model._x = x
    model._t = t
    
    # Objective function: maximize preference satisfaction
    model.setObjective(
        gp.quicksum(w[s, c] * x[s, c] for (s, c) in valid_pairs),
        GRB.MAXIMIZE  
    )

    # Constraint 1: Each student assigned to at most one school
    for s in students:
        model.addConstr(
            gp.quicksum(x[s, c] for (s2, c) in valid_pairs if s2 == s) <= 1,
            name=f"StudentLimit_{s}"
        )

    # Constraint 2: Each school's capacity
    for c in schools:
        model.addConstr(
            gp.quicksum(x[s, c2] for (s, c2) in valid_pairs if c2 == c) <= capacities[c] + t[c],
            name=f"Capacity_{c}"
        )

    # Constraint 3: Total expansion limited by budget
    model.addConstr(
        gp.quicksum(t[c] for c in schools) <= B,
        name="TotalExpansion"
    )
    
    # Constraint 4: Aggregated linearized stability constraints
    for s in students:
        s_prefs = student_preferences.get(s, {})
        
        # for each school in this student's preferences
        for s_rank, c in s_prefs.items():
            if (s, c) not in valid_pairs:
                continue
    
            # find schools that student s prefers more than c
            better_schools_for_s = [
                sch for r, sch in s_prefs.items() 
                if int(r) < int(s_rank) and (s, sch) in valid_pairs
            ]
            
            # find students that school c prioritizes over student s
            if c in school_priorities and s in school_priorities[c]:
                c_priorities = school_priorities[c]
                s_pos_in_c = c_priorities.index(s)
                better_students_for_c = [
                    sp for sp in c_priorities[:s_pos_in_c] 
                    if (sp, c) in valid_pairs
                ]
                
                if better_students_for_c:
                    # create aggregated auxiliary variable β_s^c
                    beta = model.addVar(vtype=GRB.INTEGER, lb=0, name=f"beta_{s}_{c}")
                    
                    # higher priority students assigned to c
                    higher_priority_assigned = gp.quicksum(
                        x[sp, c] for sp in better_students_for_c
                    )
                    
                    # linearization constraints for β_s^c
                    # aggregate the effect of expansion t[c]
                    # on all higher priority assignments to c
                    
                    # Constraint: β_s^c lower bound
                    model.addConstr(
                        beta >= higher_priority_assigned - capacities[c],
                        name=f"beta_lower_{s}_{c}"
                    )
                    
                    # Constraint: β_s^c upper bounds
                    model.addConstr(
                        beta <= t[c],
                        name=f"beta_upper1_{s}_{c}"
                    )
                    
                    model.addConstr(
                        beta <= higher_priority_assigned,
                        name=f"beta_upper2_{s}_{c}"
                    )
                    
                    # var for "school c filled with higher priority students"
                    is_filled_var = model.addVar(vtype=GRB.BINARY, name=f"filled_{s}_{c}")
                    
                    # set is_filled_var using linearized constraints
                    big_M = len(better_students_for_c)
                    
                    # if is_filled_var = 1, then school is filled with higher priority students
                    model.addConstr(
                        higher_priority_assigned >= capacities[c] + beta - big_M * (1 - is_filled_var),
                        name=f"FilledIf_{s}_{c}"
                    )
                    
                    # if school is not filled with higher priority students, then is_filled_var = 0
                    model.addConstr(
                        higher_priority_assigned <= capacities[c] + beta - 1 + big_M * is_filled_var,
                        name=f"NotFilledIf_{s}_{c}"
                    )
                else:
                    # no higher priority students, so this condition can't be satisfied
                    is_filled_var = model.addVar(vtype=GRB.BINARY, name=f"filled_{s}_{c}")
                    model.addConstr(is_filled_var == 0, name=f"NoHigherPriority_{s}_{c}")
                
                # Main stability constraint: Either:
                # 1. Student s is assigned to school c, OR
                # 2. Student s is assigned to a school they prefer more than c, OR
                # 3. School c is filled with students it prefers more than s
                model.addConstr(
                    x[s, c] + 
                    gp.quicksum(x[s, better] for better in better_schools_for_s) + 
                    is_filled_var >= 1,
                    name=f"Stability_{s}_{c}"
                )
    
    return model

def solve_linear_model_2(data, B, valid_pairs):
    """
    Solves the linearized model 2 and reports results.
    """
    start_time = time.time()
    model = create_linear_model_2(data, B, valid_pairs)
    model.optimize()

    status = model.Status
    if status == GRB.OPTIMAL:
        print("Optimal solution found.")
    elif status == GRB.SUBOPTIMAL:
        print("Suboptimal solution found.")
    elif status == GRB.INFEASIBLE:
        print("Model is infeasible.")
        return time.time() - start_time, float('inf')
    elif status == GRB.UNBOUNDED:
        print("Model is unbounded.")
        return time.time() - start_time, float('inf')
    else:
        print(f"Solver ended with status code: {status}")
        return time.time() - start_time, float('inf')

    # extract results
    x = model._x
    t = model._t
    
    success_pairs = sum(1 for (_, _), var in x.items() if var.X > 0.5)
    
    print("\n=== Result ===")
    print(f"Success pairs: {success_pairs}")
    
    # show 5 example assignments
    displayed = 0
    for (s, c), var in x.items():
        if var.X > 0.5 and displayed < 5:
            print(f"Student {s} -> School {c}")
            displayed += 1
    
    # count expansions
    total_expansion = sum(var.X for var in t.values())
    expanded_schools = sum(1 for var in t.values() if var.X > 0.5)
    
    print(f"\nExpansion used: {int(total_expansion)}/{B}")
    print(f"Schools expanded: {expanded_schools}")
    
    if expanded_schools > 0:
        print("\nTop 5 expansions:")
        expansions = [(c, var.X) for c, var in t.items() if var.X > 0.5]
        expansions.sort(key=lambda x: x[1], reverse=True)
        for c, exp in expansions[:5]:
            print(f"School {c}: +{int(exp)} seats")
    
    # objective value
    objective_value = model.ObjVal
    print(f"\nObjective Value: {objective_value:.4f}")

    # metrics
    solver_time = time.time() - start_time
    mip_gap = model.MIPGap if model.SolCount > 0 else float('inf')
    
    return solver_time, mip_gap



# Results & Comparison

In [21]:
def compare_models(data, valid_pairs, budget_values):
    results = {
        'quadratic': [],
        'linear_1': [],
        'linear_2': []
    }
    
    for B in budget_values:
        quad_time, quad_gap = solve_quadratic_model(data, B, valid_pairs)
        results['quadratic'].append({
            'budget': B,
            'time': quad_time,
            'gap': quad_gap
        })
        
        lin1_time, lin1_gap = solve_linear_model_1(data, B, valid_pairs)
        results['linear_1'].append({
            'budget': B,
            'time': lin1_time,
            'gap': lin1_gap
        })
        
        lin2_time, lin2_gap = solve_linear_model_2(data, B, valid_pairs)
        results['linear_2'].append({
            'budget': B,
            'time': lin2_time,
            'gap': lin2_gap
        })
    
    return results


if __name__ == "__main__":
    data = dataset_analysis(verbose=False)
    valid_pairs = data['valid_pairs']

    budget_values = [0, 25, 50, 75, 100, 125, 150, 175, 200]
    
    results = compare_models(data, valid_pairs, budget_values)
    
    print("\n===== SOLVER TIME COMPARISON (seconds) =====")
    print(f"{'Budget':<10} {'Quadratic':<15} {'Linear 1':<15} {'Linear 2':<15}")
    print("-" * 55)
    
    for i, B in enumerate(budget_values):
        q_time = results['quadratic'][i]['time']
        l1_time = results['linear_1'][i]['time']
        l2_time = results['linear_2'][i]['time']
        print(f"{B:<10} {q_time:<15.2f} {l1_time:<15.2f} {l2_time:<15.2f}")
    
    # average times and speedups
    avg_q = sum(r['time'] for r in results['quadratic']) / len(budget_values)
    avg_l1 = sum(r['time'] for r in results['linear_1']) / len(budget_values)
    avg_l2 = sum(r['time'] for r in results['linear_2']) / len(budget_values)
    
    print("\n===== SUMMARY =====")
    print(f"Average times: Quadratic={avg_q:.2f}s, Linear1={avg_l1:.2f}s, Linear2={avg_l2:.2f}s")
    print(f"Speedup Linear1/Quadratic: {avg_q/avg_l1:.2f}x")
    print(f"Speedup Linear2/Quadratic: {avg_q/avg_l2:.2f}x")

Optimal solution found.

=== Result ===
Success pairs: 1237
Student 5b86312c093da070f20f8e13 -> School 8454_401000000222_REG
Student 5b9a60df565ddd00015e851c -> School 24329_401000000122_PRI
Student 5b86307a093da070f20c4a75 -> School 8450_401000000133_PRI
Student 5b863155093da070f210497b -> School 8421_401000000133_PRI
Student 5b893e26db64c6000197e63a -> School 8436_401000000133_REG

Expansion used: 0/0
Schools expanded: 0

Objective Value: 6812.0000
Optimal solution found.

=== Result ===
Success pairs: 1237
Student 5b86312c093da070f20f8e13 -> School 8454_401000000222_REG
Student 5b9a60df565ddd00015e851c -> School 24329_401000000122_PRI
Student 5b86307a093da070f20c4a75 -> School 8450_401000000133_PRI
Student 5b863155093da070f210497b -> School 8421_401000000133_PRI
Student 5b893e26db64c6000197e63a -> School 8436_401000000133_REG

Expansion used: 0/0
Schools expanded: 0

Objective Value: 6812.0000
Optimal solution found.

=== Result ===
Success pairs: 1247
Student 5b86312c093da070f20f8e